In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.fact_contracts
USING DELTA
AS
SELECT
    contract_transaction_unique_key,
    contract_award_unique_key,
    award_id_piid,
    modification_number,
    transaction_number,
    action_date,
    federal_action_obligation,
    total_dollars_obligated,
    parent_award_agency_id,
    parent_award_agency_name
FROM workspace.default.contracts_clean
WHERE contract_transaction_unique_key IS NOT NULL
  AND contract_award_unique_key IS NOT NULL
  AND federal_action_obligation IS NOT NULL
  AND parent_award_agency_id IS NOT NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.agency_dimension AS
SELECT DISTINCT
    parent_award_agency_id,
    parent_award_agency_name,
    CASE
        WHEN parent_award_agency_name LIKE '%DEF%' THEN 'Defense'
        WHEN parent_award_agency_name LIKE '%EDU%' THEN 'Education'
        ELSE 'Other'
    END AS agency_group
FROM workspace.default.fact_contracts;

-- Join to fact table
SELECT f.*, d.agency_group
FROM workspace.default.fact_contracts f
LEFT JOIN workspace.default.agency_dimension d
ON f.parent_award_agency_id = d.parent_award_agency_id;
